### **1.1.3. Representation of Circuits in Qiskit**

The primary class used to represent circuits in Qiskit is QuantumCircuit. This class can represent both abstract and physical circuits. More information about the QuantumCircuit class and its many methods can be found in the class documentation.

Within a QuantumCircuit, the following components can be found:

* **Qubits:** Quantum bits.
* **Classical bits:** Classical computer bits.
* **Compile-time parameters:** Information on how the circuit should be compiled.
* **Real-time variables:** Variables used during the execution of the circuit.
* **Other tracking information:** Information related to the data and parameters processed by the circuit.

Additionally, within a QuantumCircuit, there exists an array of CircuitInstructions, which encompass specific operations (gates, measurements, etc.) and the qubits and classical bits on which these operations are applied.

**Let's delve a bit deeper now.**

### a. Bits and Registers

**Qubits and Classical Bits**
- Qubits (Quantum Bit) and classical bits (Classical Bit) are represented in Qiskit by a common base class named Bit. This class itself is not directly used but serves as a base for other types of bits.
- Qubit: Class representing a quantum bit.
- Clbit: Class representing a classical bit.
- AncillaQubit: Class representing an ancillary quantum bit, a special type of qubit.

**Usage of Bits**
- Qubit and Clbit objects can be directly created by users, for example Qubit() or Clbit().
- Bits that are the same Python object or have the same index within the same register are considered equal.

**Registers**
A register is a collection of a series of bits (qubits or classical bits). Registers are used to group bits of the same type together.
- QuantumRegister: Consists of a series of qubits.
- ClassicalRegister: Consists of a series of classical bits.
- AncillaRegister: Used for holding ancillary qubits (less commonly used).

**Usage of Registers**
- Registers can be created to contain a specified number of bits, for example QuantumRegister(5) creates a register containing five qubits.
- Alternatively, registers can be created with a list of pre-existing bits.
- Providing neither the size nor the list of bits when creating a register, or providing both simultaneously, will result in an error.

**Functions and Properties of Registers**
- index: Finds the index of a given bit within this register.
- name: Gets the name of the register.
- size: Gets the size of the register.

**Usage of Registers in Circuits**
- Registers are used within QuantumCircuit to group qubits and classical bits.
- Registers can appear with a name in circuit visualizations and exports to other languages (e.g., QuantumCircuit.draw(), qasm2, and qasm3).
- Classical bits and registers are the fundamental way of representing classical data in Qiskit and remain the most supported method.
- Long-term, Qiskit is moving towards a more robust and comprehensive data model for representing classical data.

### b. Instruction Contents

Each element of the `QuantumCircuit.data` array is a CircuitInstruction object, also known as an "instruction context". This object is a data class representing instructions within a QuantumCircuit, detailing what operations (operations) are to be performed and on which data (qubits and clbits). We'll delve deeper into this in subsequent sections.

The **CircuitInstruction class** represents instructions contained within a QuantumCircuit. This class is written in Rust, a programming language known for its performance and safety advantages. Changes made to instances (objects) of the CircuitInstruction class do not directly reflect in the QuantumCircuit circuit. In other words, if you modify an instance of this class, these changes won't automatically appear in the circuit. You cannot directly modify instructions within the circuit through the QuantumCircuit interface. Instead, circuits are created and stored via the QuantumCircuit interface. To modify instructions within the circuit, you need to use an intermediate representation called DAGCircuit in conjunction with a transpiler.

The **DAGCircuit** is an intermediate representation used to facilitate easier manipulation of circuits.

Methods that add instructions to QuantumCircuit (such as append() and other standard gate methods) return an InstructionSet that points to multiple CircuitInstructions simultaneously. An InstructionSet is a collection of instructions and their contexts. However, InstructionSet is not widely used in Qiskit anymore. It provides limited methods to modify instructions after they have been added and is no longer recommended. Instead, alternative methods are suggested.

**In summary:**

- CircuitInstruction is an object that specifies what to do and on which data within a circuit.
- This object is implemented in Rust and is not directly modifiable.
- When adding instructions to QuantumCircuit, methods return an InstructionSet pointing to multiple instructions.
- InstructionSet is less commonly used now, and alternative methods are recommended.

### c. Operations, Instructions, and Gates

Each operation within CircuitInstruction is fundamentally referred to as an Operation. This term represents the simplest structure that specifies any action to be performed within a circuit, such as applying a gate or performing a measurement. Every operation utilizes this basic structure when integrated into a circuit. Thus, the Operation interface abstractly and generally represents operations conducted within circuits. It is used in theoretical or abstract circuits, allowing direct addition of mathematical operations or objects to circuits. For example, you can directly add a mathematical object like a quantum operator to a circuit.

**1. Operation()**

It is a structure or class that defines operations achievable in a quantum computer.

This structure encompasses various operations conducted in quantum circuits. Like all operations in physical circuits, most operations are instances of the more concretely defined "Instruction" class. It represents instructions that can be locally executed by a quantum processing unit (QPU). For example, an instruction such as "Measure" is an operation that can be locally executed by the quantum processor. Such instructions do not necessarily need to be unitary (i.e., invertible). Operations that are unitary are referred to as "Gate" operations. In other words, instructions referred to as "Gate" operations are unitary operations, whereas instructions like "Measure" may not be unitary.

**2. Instruction**

It is  the general quantum operations.

Instructions operate on Qubits and Clbits and are defined with specific parameters (params). This means that instructions affect only specific quantum bits (Qubits) and classical bits (Clbits) that are defined for them. An instruction must work only with the Qubits and Clbits specified for it, without depending on other parts or components of the circuit. Additionally, instructions should not contain hidden or unspecified parameters; all parameters should be explicitly defined. This makes instructions easier to understand, manage, and reuse.

An Instruction can be decomposed into other circuits using the Instruction.definition property. This means defining smaller parts of the circuit. These definitions can use the most suitable set of operations for you, provided that the gates included in the circuit have a specific order. If you don't want an instruction to be optimized, the definition can be set to None.

Technically, the parameters of an instruction can be anything, but it's generally best to use real numbers. Qiskit sometimes breaks this rule and uses parameters of different types, which can lead to errors. If your instruction is defined with angles, you can use parameters at compile time and work well with QuantumCircuit.assign_parameters().

An Instruction does not need to be unitary (invertible), but its subclass Gate is unitary and includes additional methods like to_matrix() and control().



**3. Gate**

It represents gates in quantum operations. It is a unitary operation.

A Gate is a unitary operation used in quantum circuits that operates solely on quantum bits (Qubits) and does not interact with classical bits. Gates utilize all the properties of the Instruction class, meaning they have the same methods and parameters. Each Gate should have a fundamental definition: Every Gate must have a basic definition that explains how it operates. This definition shows what the gate does and how it functions. These definitions are predefined if you use a gate within Qiskit. If you create our own gate, it should have this definition

###d. Real-Time Classical Computation

Let's start by explaining the abbreviation QPU, which will be used in the following definitions:

**QPU:**
- Stands for Quantum Processing Unit.
- It is the hardware component that performs computations in quantum computers.
- While processors (CPUs) handle computation tasks in traditional computers, QPUs perform these tasks in quantum computers.
- QPUs leverage quantum mechanical properties such as superposition and entanglement using quantum bits (qubits). This allows them to perform certain types of calculations much faster and more efficiently than traditional computers.

Qiskit supports real-time classical computations that occur during QPU execution and influence its results. However, hardware support in this regard is still under development.

Qiskit uses expression (Expr) and type systems to manage real-time classical computations. Here’s how these systems work:

- **Expr and Type System:** Qiskit uses Expr and Type to represent classical computations. Expr defines the computation operations, and Type specifies what kind of data these expressions operate on.

- **Expr Tree:** Real-time computations are organized in an "Expr Tree" structure. This structure consists of nodes where each Expr is a node. It shows how each Expr is related to others and in what order operations are performed.

- **Expr Node:** Each expression is represented as a node called "Expr." These nodes are part of computation operations, and each node is associated with a specific type. Types define what the data is and how it is processed.

In summary, real-time classical computations in Qiskit are defined using Expr and type systems. These Exprs are organized as nodes in an Expr tree, where each node is associated with a specific data type. This system determines how computations are performed and what data is used.


###e. Control Flow in Circuits

Within QuantumCircuit, classical control flow is represented by specific Instructions that are subclasses of ControlFlowOp. Let's examine them here.

**ControlFlowOp**

**ControlFlowOp** is an abstract class that theoretically encompasses all control flow operations. These control flow operations (IfElseOp, WhileLoopOp, SwitchCaseOp, and ForLoopOp) have specific branching conditions and strategies. Additionally, they store all sub-circuit blocks they contain in a blocks property. In other words, each control flow operation has specific conditions that determine when and how it branches or loops, and it also holds sub-circuits that can be executed within these operations in the blocks property.

- **IfElseOp**

A circuit operation that evaluates a given condition and executes one program (true_body) if the condition evaluates to true, and optionally another program (false_body) if specified.

- **WhileLoopOp**

A circuit operation that repeatedly executes a sub-circuit (body) until a condition (condition) evaluates to false.

- **SwitchCaseOp**

A circuit operation that executes a specific circuit block based on matching a target against a sequential list of values.

- **ForLoopOp**

A circuit operation that iteratively executes a sub-circuit (body) parameterized with a parameter (loop_parameter) across a given range of integer values (indexset).

ControlFlowOp subclasses are also subclasses of Instruction. This means that when stored within circuit instructions (CircuitInstruction), they are applied on specific qubits and clbits. This can lead to some issues related to data consistency. Qubits and Clbit objects used within the sub-circuit blocks of control flow operations may not correspond to the relevant objects in CircuitInstruction. Code using control flow operations in Qiskit should be aware of this.
